In [2]:
# importando pacotes utilizados

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as re
import time
import plotly.express as px

In [3]:
#estrutura inicial dos dados
url_nba = f'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2021-22&SeasonType=Regular%20Season&StatCategory=PTS' # url da api com f-string para iterar listas
dados = re.get(url=url_nba).json() # fazendo request do json

colunas_iniciais = ['YEAR', 'SEASON'] + dados['resultSet']['headers'] # estrutura inicial do dataframe

df_final = pd.DataFrame(columns=colunas_iniciais) # iniciando o dataframe

# parametros api
tipo_season = ['Regular%20Season', 'Playoffs'] # tipo de temporada
year = ['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21','2021-22', '2022-23'] # ano da temporada

for y in year: # iterando lista de anos de temporada
    for t in tipo_season: # iterando tipos de temporada playoff ou regular
        url_nba = f'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season={y}&SeasonType={t}&StatCategory=PTS' # url da api com f-string para iterar listas
        dados = re.get(url=url_nba).json() # fazendo request do json
        df1 = pd.DataFrame(dados['resultSet']['rowSet'], columns=dados['resultSet']['headers']) # request da informacao selecionada na api anteriormente
        df2 = pd.DataFrame({'YEAR':[y for i in range(len(df1))], # aplicando o ano e a temporada para cada linha 
                            'SEASON':[t for i in range(len(df1))]})
        df3 = pd.concat([df2, df1], axis=1) # concatenando em um dataframe ajustado
        df_final = pd.concat([df_final, df3], axis = 0) # empilhando as bases
        print(f'Dados obtidos: ano - {y}, temporada - {t}')
        #time.sleep(np.random.uniform(low=5, high = 10))
        
print ('Executado')


Dados obtidos: ano - 2010-11, temporada - Regular%20Season
Dados obtidos: ano - 2010-11, temporada - Playoffs
Dados obtidos: ano - 2011-12, temporada - Regular%20Season
Dados obtidos: ano - 2011-12, temporada - Playoffs
Dados obtidos: ano - 2012-13, temporada - Regular%20Season
Dados obtidos: ano - 2012-13, temporada - Playoffs
Dados obtidos: ano - 2013-14, temporada - Regular%20Season
Dados obtidos: ano - 2013-14, temporada - Playoffs
Dados obtidos: ano - 2014-15, temporada - Regular%20Season
Dados obtidos: ano - 2014-15, temporada - Playoffs
Dados obtidos: ano - 2015-16, temporada - Regular%20Season
Dados obtidos: ano - 2015-16, temporada - Playoffs
Dados obtidos: ano - 2016-17, temporada - Regular%20Season
Dados obtidos: ano - 2016-17, temporada - Playoffs
Dados obtidos: ano - 2017-18, temporada - Regular%20Season
Dados obtidos: ano - 2017-18, temporada - Playoffs
Dados obtidos: ano - 2018-19, temporada - Regular%20Season
Dados obtidos: ano - 2018-19, temporada - Playoffs
Dados obti

## Glossary
* GP - Games Played
* MIN - Minutes Played
* PTS - Points
* FGM - Field Goals Made
* FGA - Field Goals Attempted
* FG% - Field Goal Percentage
* 3PM - 3 Point Field Goals Made
* 3PA - 3 Point Field Goals Attempted
* 3P% - 3 Point Field Goal Percentage
* FTM - Free Throws Made
* FTA - Free Throws Attempted
* FT% - Free Throw Percentage
* OREB - Offensive Rebounds
* DREB - Defensive Rebounds
* REB - Rebounds
* AST - Assists
* STL - Steals
* BLK - Blocks
* TOV - Turnovers

In [7]:
df_celtics = df_final[df_final['TEAM'] == 'BOS']

### Por ano/temporada, qual o jogador com maior qtd. de jogos

In [17]:
df_celtics

,YEAR,SEASON,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
18,2010-11,Regular%20Season,1718,19,Paul Pierce,1610612738,BOS,80,2774,507,...,430,260,80,51,171,231,1511,1584,1.52,0.47
32,2010-11,Regular%20Season,951,33,Ray Allen,1610612738,BOS,80,2890,480,...,274,219,77,16,117,147,1321,1266,1.87,0.66
69,2010-11,Regular%20Season,708,69,Kevin Garnett,1610612738,BOS,71,2220,434,...,631,171,95,57,115,148,1056,1477,1.49,0.83
86,2010-11,Regular%20Season,201145,86,Jeff Green,1610612738,BOS,75,2428,368,...,360,108,53,37,99,210,998,959,1.09,0.54
98,2010-11,Regular%20Season,201175,99,Glen Davis,1610612738,BOS,78,2298,359,...,425,92,78,30,85,231,915,943,1.08,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2022-23,Playoffs,1630573,129,Sam Hauser,1610612738,BOS,15,104,10,...,17,4,1,2,0,8,30,35,0.00,0.00
148,2022-23,Playoffs,1628436,148,Luke Kornet,1610612738,BOS,8,32,7,...,10,0,0,1,0,2,17,27,0.00,0.00
159,2022-23,Playoffs,203488,159,Mike Muscala,1610612738,BOS,6,21,2,...,4,1,0,0,1,1,9,10,1.00,0.00
189,2022-23,Playoffs,1630551,185,Justin Champagnie,1610612738,BOS,4,10,1,...,0,0,0,0,0,2,2,-1,0.00,0.00


In [19]:
df_celtics.groupby(['YEAR', 'SEASON', 'PLAYER'], sort=False)['GP'].max().reset_index()

,YEAR,SEASON,PLAYER,GP
0,2010-11,Regular%20Season,Paul Pierce,80
1,2010-11,Regular%20Season,Ray Allen,80
2,2010-11,Regular%20Season,Kevin Garnett,71
3,2010-11,Regular%20Season,Jeff Green,75
4,2010-11,Regular%20Season,Glen Davis,78
...,...,...,...,...
382,2022-23,Playoffs,Sam Hauser,15
383,2022-23,Playoffs,Luke Kornet,8
384,2022-23,Playoffs,Mike Muscala,6
385,2022-23,Playoffs,Justin Champagnie,4


In [ ]:
fig = px.bar(df_playoffs, x='PLAYER', y = 'REB')
fig.show()